<img src="./images/DLI_Header.png" style="width: 400px;">

# 2.0 GitOps using ArgoCD

In this notebook, you'll explore an already set up Argo CD instance, connecting it to a Git repository, adding other repositories (such as Helm), and deploying a simple application using GitOps.



## ArgoCD

<center><img src="./images-dli/argocd_architecture.png" style="width: 400px;"></center>

Argo CD is a declarative, GitOps continuous delivery tool for Kubernetes. It automates the deployment and lifecycle management of applications in Kubernetes clusters by using Git repositories as the single source of truth. This methodology reduces hands on changes within a cluster helping reduce human error and maintain auditability of cluster changes.  

**Key Features of Argo CD**
- GitOps-based Deployment: Ensures that the desired state of applications is always synchronized with the Git repository.
- Declarative Application Management: Supports Kubernetes manifests in Helm, Kustomize, Jsonnet, and plain YAML.
- Automated Synchronization: Continuously monitors and applies updates from Git.
- Multi-Cluster Support: Manages deployments across multiple Kubernetes clusters.
- RBAC and SSO Integration: Supports authentication and authorization via OIDC, GitHub, LDAP, etc.
- Web UI & CLI: Provides an intuitive UI and CLI for managing applications.


## 2.1 Check the ArgoCD status

We already have deployed argocd by default for you using helm chart with the following commands:
```python
    kubectl create ns argocd
    helm repo add argo https://argoproj.github.io/argo-helm
    helm repo update
    helm upgrade --install argocd -n argocd argo/argo-cd --version 6.7.18 -f /dli/values.yaml
```

Lets check its status: 

### 2.1.1 Get Pods of ArgoCD

Example output: 

```
NAME                                               READY   STATUS    RESTARTS   AGE
argocd-application-controller-0                    1/1     Running   0          31m
argocd-applicationset-controller-d86f6bb77-rk2ml   1/1     Running   0          31m
argocd-dex-server-846bb57578-9sxhk                 1/1     Running   0          31m
argocd-notifications-controller-79c45548f9-85snv   1/1     Running   0          31m
argocd-redis-55d588f877-rslf7                      1/1     Running   0          31m
argocd-repo-server-6546679994-j8hs6                1/1     Running   0          31m
argocd-server-79f597f4ff-4bx6k                     1/1     Running   0          31m
```

In [ ]:
!kubectl get pods -n argocd

### 2.1.2 Get Services of ArgoCD

In Kubernetes we use services to expose our application pods to the network.

Example output: 

```
NAME                               TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)                      AGE
argocd-applicationset-controller   ClusterIP   10.96.229.240    <none>        7000/TCP                     32m
argocd-dex-server                  ClusterIP   10.108.255.86    <none>        5556/TCP,5557/TCP            32m
argocd-redis                       ClusterIP   10.107.176.169   <none>        6379/TCP                     32m
argocd-repo-server                 ClusterIP   10.97.56.97      <none>        8081/TCP                     32m
argocd-server                      NodePort    10.110.251.146   <none>        80:30080/TCP,443:30443/TCP   32m
```

As you can see from above, we have two different types in our lab, ClusterIP and NodePort. ClusterIP exposes the service within our cluster to other services in that namespace, whereas our NodePort is used to expose our service on the Nodes IP address. You will also note we have port mappings, so for our argocd-server where we are enabling it to be accessed from the Nodes IP, we are mapping port 30080 to the pods port of 80. 

In [ ]:
!kubectl get svc -n argocd

### 2.1.3 Check if ArgoCD is ready

In [ ]:
import subprocess
import time

while True:
    result = subprocess.run(
        ["kubectl", "rollout", "status", "deployment/argocd-server", "-n", "argocd", "--timeout=5s"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    if "successfully rolled out" in result.stdout:
        print("ArgoCD server is ready!")
        break
    else:
        print("Waiting for ArgoCD server to be ready...")
        time.sleep(5)


## 2.2 Accessing Argo CD

Here we will 
- Obtain the Argo CD UI URL and login credentials.
- Log into the Argo CD web UI and explore its features:
    - Dashboard overview
    - Applications view
    - Settings and project configurations
    

### 2.2.1 Get Argocd login credentials

First time you can login with **username: admin** and the random password generated during the installation. 

You can find the password by running:

```
kubectl -n argocd get secret argocd-initial-admin-secret -o jsonpath="{.data.password}" | base64 -d
```

In [ ]:
!kubectl -n argocd get secret argocd-initial-admin-secret -o jsonpath="{.data.password}" | base64 -d

### 2.2.2 Interact with the ArgoCD Web UI

<center><img src="./images-dli/argocd_ui.png" style="width: 800px;"></center>




In [ ]:
import subprocess

subprocess.Popen(
    ["kubectl", "-n", "argocd", "port-forward", "--address", "0.0.0.0", "service/argocd-server", "30009:80"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
    close_fds=True
)

### 2.2.3 Get the ArgoCD Web UI URL


<div class="alert alert-block alert-warning">
After logging into ArgoCD UI, check the URL, it would be something like `*/argocd/argocd/*`, please remove one extra `/argocd` from it. 

</div>

In [ ]:
%%js
const href = window.location.hostname;
let a = document.createElement("a");
let link = document.createTextNode('Open ArgoCD UI!');
a.appendChild(link);
a.href = "http://" + href + "/argocd";
a.style.cssText = "text-decoration: none; color: white; font-size: 20px; font-weight: bold; padding: 15px 25px; background-color: #76B900; border-radius: 8px; display: inline-block; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.2);";
a.target = "_blank";
a.style.color = "white";
a.onmouseover = () => a.style.backgroundColor = "#5a9300";
a.onmouseout = () => a.style.backgroundColor = "#76B900";
element.appendChild(a);

## 2.3 Interact with ArgoCD WebUI

### 2.3.1 ArgoCD Web UI Application View
<center><img src="./images-dli/argocd_ui_application.png" style="width: 800px;"></center>
In Argo CD, an Application is a representation of a deployed workload in a Kubernetes cluster. It defines how a set of Kubernetes manifests (from a Git repository, Helm chart, or Kustomize) should be deployed and managed.

By selecting the Applications View from the left hand pane, ArgoCD displays all managed applications, their status, and sync status.
Each application shows:
- Health Status (e.g., Healthy, Degraded, Suspended)
- Sync Status (e.g., Synced, OutOfSync)
- Namespace & Destination Cluster
  
Clicking an application opens a detailed view, showing:

- Live resource tree visualization
- Sync history and rollback options
- Logs and manifest comparisons    


### 2.3.2 ArgoCD Web UI Settings View

<center><img src="./images-dli/argocd_ui_settings_view2.png" style="width: 800px;"></center>

Settings View (Repositories Management): Found under Settings → Repositories
- Lists connected Git repositories, Helm repositories, and other sources.
- Allows adding new repositories by specifying:
    - Repository URL
    - Authentication credentials (if needed)
    - Repository type (Git, Helm, Kustomize, etc.)


## 2.4 Create a GitHub repository

1. Go to https://github.com
2. login to your account, or create one if you don't have.
3. Create a new **private** repository and call it `llmops-nvidia`.
  

<div class="alert alert-block alert-warning">
Before proceeding, have the git repo created. It would be something like: 

https://github.com/github-username/llmops-nvidia

</div>

<center><img src="./images-dli/llmops-git-repo.png" style="width: 800px;"></center>


## 2.5 Add a Deploy SSH Key to the GitHub repository

Here we will be creating a SSH key and adding to your Git repository for commiting.

### 2.5.1 Create SSH Key

sample output: 

```
Generating public/private ed25519 key pair.
Your identification has been saved in /root/.ssh/id_ed25519
Your public key has been saved in /root/.ssh/id_ed25519.pub

```

In [ ]:
!ssh-keygen -t ed25519 -C "llmops-nvidia" -f ~/.ssh/id_ed25519 -N ""

### 2.5.2 Copy SSH key

In [ ]:
!cat ~/.ssh/id_ed25519.pub

<div class="alert alert-block alert-warning">

Copy the above output public key
</div>

### 2.5.3 Add SSH key to your GitHub account. 

1. In the line of tabs under the repository name, click `Settings`.
2. In the left sidebar, click `Deploy keys`.
3. Click `Add deploy key`.
4. In the `Title` field, add a descriptive label for the new key. For example, "llmops-nvidia".
5. In the `Key` field, paste your public key.
6. Make sure to check `Allow write access`.
7. Click `Add key`.
8. If prompted, confirm access to your account on GitHub.

<center><img src="./images-dli/deploy-key.png" style="width: 800px;"></center>


## 2.6 Clone the llmops-nvidia repository

<div class="alert alert-block alert-warning">

Add the git Username
</div>

In [ ]:
print("Please enter your Git username:")
git_username = input()

# Add assertions to validate the variables
assert git_username and git_username.strip(), "GitHub username cannot be empty"


In [ ]:
git_repo_name="llmops-nvidia"
git_base_url="github.com"

In [ ]:
git_repo_url=f"git@{git_base_url}:{git_username}/{git_repo_name}.git" 
git_repo_url_ssh=f"ssh://git@{git_base_url}/{git_username}/{git_repo_name}.git"
commit_name=git_username 
commit_email=f"{git_username}@llmops-nvidia"
print(git_repo_url)

In [ ]:
!ssh-keyscan github.com >> ~/.ssh/known_hosts

Sample output: 

```
Cloning into 'llmops-nvidia'...
warning: You appear to have cloned an empty repository.
```

In [ ]:
!git clone $git_repo_url llmops-nvidia

In [ ]:
!find ./llmops-nvidia

<div class="alert alert-info">

You should have an empty folder named llmops-nvidia
</div>
    


## 2.7 Create Example ArgoCD Application


<center><img src="./images-dli/example_flow.png" style="width: 1000px;"></center>


<div class="alert alert-warning">

For argocd applications, we need GIT repo url in following manner: 

ssh://git@github.com/username/llmops-nvidia.git'

</div>

### 2.7.1 Add Example K8s manifests

#### 2.7.1.1 Create Folder for the K8s manifests

In [ ]:
!mkdir -p llmops-nvidia/k8s-manifests

#### 2.7.1.2 Create Folder for the example application 

We will create a sample application for deploying `nginx`

In [ ]:
!mkdir -p llmops-nvidia/k8s-manifests/nginx

#### 2.7.1.3 Add K8s manifest for the example application (nginx)

We add here the Kubernetes manifest for the application: 

1. **Deployment**
   
Deployment is a good fit for managing a stateless application workload on your cluster, where any Pod in the Deployment is interchangeable and can be replaced if needed.

Creating a deployment with following configuration:
```
name nginx-deployment
container image: nginx
replicas: 1
```

2. **Service**
   
In Kubernetes, a Service is a method for exposing a network application that is running as one or more Pods in your cluster. The Service API, part of Kubernetes, is an abstraction to help you expose groups of Pods over a network. Each Service object defines a logical set of endpoints (usually these endpoints are Pods) along with a policy about how to make those pods accessible.

Kubernetes Service types allow you to specify what kind of Service you want. The available type values and their behaviors are:

- **ClusterIP**: Exposes the Service on a cluster-internal IP. Choosing this value makes the Service only reachable from within the cluster. This is the default that is used if you don't explicitly specify a type for a Service. You can expose the Service to the public internet using an Ingress or a Gateway.

- **NodePort**: Exposes the Service on each Node's IP at a static port (the NodePort). 

- **LoadBalancer**: Exposes the Service externally using an external load balancer. Kubernetes does not directly offer a load balancing component; you must provide one, or you can integrate your Kubernetes cluster with a cloud provider.

- **ExternalName**: Maps the Service to the contents of the externalName field (for example, to the hostname api.foo.bar.example). The mapping configures your cluster's DNS server to return a CNAME record with that external hostname value. No proxying of any kind is set up.


 

In [ ]:
# We create yaml files and save them

nginx_deployment_yaml = f"""
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:latest
        ports:
        - containerPort: 80
"""

nginx_service_yaml = f"""
apiVersion: v1
kind: Service
metadata:
  name: nginx-service
spec:
  selector:
    app: nginx
  ports:
  - protocol: TCP
    port: 80
    targetPort: 80
  type: ClusterIP
"""
with open("llmops-nvidia/k8s-manifests/nginx/deployment.yaml", "w") as f:
    f.write(nginx_deployment_yaml)
with open("llmops-nvidia/k8s-manifests/nginx/service.yaml", "w") as f:
    f.write(nginx_service_yaml)

### 2.7.2 Create application for tracking example NGINX  in ArgoCD

In [ ]:
!mkdir -p llmops-nvidia/applications/nginx

In [ ]:
argocd_nginx_application_yaml = f"""
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: nginx
  namespace: argocd
spec:
  destination:
    namespace: nginx
    server: 'https://kubernetes.default.svc'
  source:
    path: k8s-manifests/nginx
    repoURL: '{git_repo_url_ssh}'
    targetRevision: main
    directory:
      recurse: true
  project: default
  syncPolicy:
    syncOptions:
    - Validate=false
    - CreateNamespace=true
    automated:
      prune: true
      selfHeal: true
      allowEmpty: false
"""
with open("llmops-nvidia/applications/nginx/app.yaml", "w") as f:
    f.write(argocd_nginx_application_yaml)

### 2.7.3 Check directory structure now

We now have the base to commit
Example output

```
llmops-nvidia/
├── applications
│   └── nginx
│       └── app.yaml
└── k8s-manifests
    └── nginx
        ├── deployment.yaml
        └── service.yaml
```

In [ ]:
!tree llmops-nvidia/

### 2.7.4 Commit the code


#### 2.7.4.1 Configure the commit user

In [ ]:
!git config --global user.email $commit_email
!git config --global user.name $commit_name

#### 2.7.4.2 Add, commit and push to git

We are directly commiting to main, but ideally you would want to create a branch and merge. But for this lab, we commit directly to git

Sample output: 

```
main (root-commit) 4f5f53b] add base argocd app code
 3 files changed, 58 insertions(+)
 create mode 100644 applications/nginx/app.yaml
 create mode 100644 k8s-manifests/nginx/deployment.yaml
 create mode 100644 k8s-manifests/nginx/service.yaml
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 96 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (9/9), 1.07 KiB | 22.00 KiB/s, done.
Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
To github.com:<username>/llmops-nvidia.git
 * [new branch]      main -> main
```

In [ ]:
!cd llmops-nvidia/ && git add . && git commit -m "add base argocd app code" && git push

### 2.7.5 Check GitHub UI for the repo

<center><img src="./images-dli/git_repo_first_commit.png" style="width: 800px;"></center>



## 2.8 Connecting ArgoCD to Git repo

To enable GitOps workflows, Argo CD must be connected to a Git repository that contains Kubernetes manifests, Helm charts, or Kustomize configurations. This allows Argo CD to track changes and synchronize applications automatically.


We will be using the same SSH key to connect Argo CD to the Git repository. In production, you would want to use a different key for your personal access and for the CI/CD pipeline.



### 2.8.1 Check Generated private SSH Key

In [ ]:
!cat ~/.ssh/id_ed25519

### 2.8.2 Create Secret for ArgoCD to connect to Git Repo

Argo CD needs authentication credentials to connect to a Git repository. While this can be configured via the UI under Settings → Repositories, here we will use a Kubernetes Secret to store SSH credentials.


In [ ]:
# make folder for all argocd config files
!mkdir argocd_configs

In [ ]:
with open("/root/.ssh/id_ed25519", "r") as f:
    private_key = f.read().strip()  # Read the key as-is, preserving newlines

# Build the YAML string properly, ensuring indentation for the SSH key
secret_yaml = f"""apiVersion: v1
kind: Secret
metadata:
  name: github-repo
  namespace: argocd
  labels:
    argocd.argoproj.io/secret-type: repository
stringData:
  url: {git_repo_url}
  sshPrivateKey: |
""" + "\n".join([f"    {line}" for line in private_key.splitlines()]) + f"""
  insecure: "false" # Do not perform a host key check for the server. Defaults to "false"
  enableLfs: "false" # Enable git-lfs for this repository. Defaults to "false"
"""

# Print the final YAML to verify formatting
print(secret_yaml)

# Save it to a file
with open("argocd_configs/argocd-ssh-secret.yaml", "w") as f:
    f.write(secret_yaml)


#### 2.8.2.1 Create Secret

In [ ]:
!kubectl apply -f argocd_configs/argocd-ssh-secret.yaml

#### 2.8.2.2 Check Created secret

Note that your secret name is: github-repo

In [ ]:
!kubectl get secret -n argocd 

#### 2.8.2.3 Check Argo CD UI for the repositories

[Open ArgoCD!](/settings/repos)

<center><img src="./images-dli/argocd_repo_connected.png" style="width: 1000px;"></center>

It should show successful

but there will be no applications even though we have added our nginx application

### 2.9 Create Application to track other Applications

We will create an argocd application which will track all applications under a certain folder. 

```
source:
    path: applications/
    repoURL: 'ssh://{git_repo_url_ssh}'
    targetRevision: main
    directory:
      recurse: true
```

In [ ]:
app_for_app_yaml = f"""
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: argocd-components
  namespace: argocd
spec:
  destination:
    namespace: argocd
    server: 'https://kubernetes.default.svc'
  source:
    path: applications/
    repoURL: '{git_repo_url_ssh}'
    targetRevision: main
    directory:
      recurse: true
  project: default
  syncPolicy:
    syncOptions:
    - Validate=false
    - CreateNamespace=true
    automated:
      prune: true
      selfHeal: true
      allowEmpty: false
"""
with open("argocd_configs/app_for_app_yaml.yaml", "w") as f:
    f.write(app_for_app_yaml)


In [ ]:
!cat argocd_configs/app_for_app_yaml.yaml

In [ ]:
!kubectl apply -f argocd_configs/app_for_app_yaml.yaml

### 2.9.1 Check Applications via CLI

Sample Output: 

```
NAMESPACE   NAME                SYNC STATUS   HEALTH STATUS
argocd      argocd-components   Synced        Healthy
argocd      nginx               Synced        Healthy
```

In [ ]:
!kubectl get applications -A

### 2.9.2 Sync vi UI
As we commit our code to Git, argocd usually sync automatically after every 5 minutes. But we can force it to sync either via UI or CLI. 

Here we use UI to sync. 

Click on the sycn button on the `argocd-components` application as shown in diagram.
<center><img src="./images-dli/sync-apps.png" style="width: 435px"></center>

#### 2.9.3 Check Applications via UI

[Open ArgoCD!](/argocd/applications)

It should show successful 2 applications

<center><img src="./images-dli/argocd-base-applications.png" style="width: 800px;"></center>


---
<h2 style="color:green;">Congratulations!</h2>

You've made it through the second Notebook. In this notebook, you have:
- Discovered and explored ArgoCD UI.
- Created a GitHub repository
- Created an example Kubernetes manifests application files.
- Created sample ArgoCD application for deploying example Kubernetes application.
- Linked GitHub repository with ArgoCD.
- Create ArgoCD application to track other ArgoCD applications.

Next, you'll see learn to deploy Nemo Microservices and its components via ArgoCD.

Move on to [03_Nemo_Microservices_Components_Deployment.ipynb](03_Nemo_Microservices_Components_Deployment.ipynb)

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>